In [19]:
import numpy as np
import pandas as pd

In [20]:
train = pd.read_csv("../data/tabular_playground/train.csv")
test = pd.read_csv("../data/tabular_playground/test.csv")

data = pd.concat([train, test], sort=False)

In [21]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1.0,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0.0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0.0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0.0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1.0,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,199995,NaN,3,"Cash, Cheryle",female,27.00,0,0,7686,10.12,NaN,Q
99996,199996,NaN,1,"Brown, Howard",male,59.00,1,0,13004,68.31,NaN,S
99997,199997,NaN,3,"Lightfoot, Cameron",male,47.00,0,0,4383317,10.87,NaN,S
99998,199998,NaN,1,"Jacobsen, Margaret",female,49.00,1,2,PC 26988,29.68,B20828,C


In [22]:
data["Sex"].replace(["male", "female"], [0,1], inplace=True)
data["Embarked"].fillna(("S"), inplace=True)
data["Embarked"] = data["Embarked"].map(
    {"S":0, "C":1, "Q":2}
).astype(int)
data["Fare"].fillna(np.mean(
    data["Fare"]
), inplace=True)

In [23]:
data["Age"].fillna(data["Age"].median(), inplace=True)

In [24]:
data["FamilySize"] = data["Parch"] + data["SibSp"] + 1
data["IsAlone"] = 0
data.loc[data["FamilySize"] == 1, "IsAlone"] = 1

In [25]:
data["FirstName"] = data.Name.map(
    lambda x: str(x.split(",")[0])
)
data["SurName"] = data.Name.map(
    lambda x: str(x.split(",")[1])
)

for col in ['Name', 'FirstName', 'SurName']:
    data["Counter_"+col] = data[col].map(
        data.groupby(col)["PassengerId"].count().to_dict()
    )

In [26]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,FirstName,SurName,Counter_Name,Counter_FirstName,Counter_SurName
0,0,1.0,1,"Oconnor, Frankie",0,31.00,2,0,209245,27.14,C12239,0,3,0,Oconnor,Frankie,1,47,51
1,1,0.0,3,"Bryan, Drew",0,31.00,0,0,27323,13.35,NaN,0,1,1,Bryan,Drew,1,54,31
2,2,0.0,3,"Owens, Kenneth",0,0.33,1,2,CA 457703,71.29,NaN,0,4,0,Owens,Kenneth,2,165,1128
3,3,0.0,3,"Kramer, James",0,19.00,0,0,A. 10866,13.04,NaN,0,1,1,Kramer,James,2,53,4736
4,4,1.0,3,"Bond, Michael",0,25.00,0,0,427635,7.76,NaN,0,1,1,Bond,Michael,1,39,3662


In [27]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin','FirstName', 'SurName']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Counter_Name,Counter_FirstName,Counter_SurName
0,NaN,3,0,19.0,0,0,63.01,0,1,1,1,18,1384
1,NaN,3,1,53.0,0,0,5.81,0,1,1,1,108,125
2,NaN,1,1,19.0,0,0,38.91,1,1,1,1,511,244
3,NaN,2,0,25.0,0,0,12.93,0,1,1,1,51,770
4,NaN,1,1,17.0,0,2,26.89,1,3,0,1,8,387


In [28]:
y_train = train["Survived"]
X_train = train.drop("Survived", axis=1)
X_test = test.drop("Survived", axis=1)

In [29]:
y_train.head()

0    1.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: Survived, dtype: float64

In [30]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Counter_Name,Counter_FirstName,Counter_SurName
0,1,0,31.00,2,0,27.14,0,3,0,1,47,51
1,3,0,31.00,0,0,13.35,0,1,1,1,54,31
2,3,0,0.33,1,2,71.29,0,4,0,2,165,1128
3,3,0,19.00,0,0,13.04,0,1,1,2,53,4736
4,3,0,25.00,0,0,7.76,0,1,1,1,39,3662


In [31]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Counter_Name,Counter_FirstName,Counter_SurName
0,3,0,19.0,0,0,63.01,0,1,1,1,18,1384
1,3,1,53.0,0,0,5.81,0,1,1,1,108,125
2,1,1,19.0,0,0,38.91,1,1,1,1,511,244
3,2,0,25.0,0,0,12.93,0,1,1,1,51,770
4,1,1,17.0,0,2,26.89,1,3,0,1,8,387


In [32]:
# lightLGM + skfold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [33]:
y_preds = []
models = []
oof_train = np.zeros((len(X_train), ))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [34]:
# categorical_features = ["Embarked", "Pclass", "Sex"]
categorical_features = ["Embarked", "Pclass", "Sex", "IsAlone"]


In [35]:
params = {
    "objective":"binary",
    "max_bin":300,
    "learning_rate":0.05,
    "num_leaves":40
}

In [36]:
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index,:]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(
        X_tr, y_tr, categorical_feature=categorical_features
    )

    lgb_eval = lgb.Dataset(
        X_val, y_val, reference=lgb_train, categorical_feature=categorical_features 
    )

    model = lgb.train(
        params, lgb_train, valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10, num_boost_round=1000, early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(
        X_val, num_iteration=model.best_iteration
    )

    y_pred = model.predict(
        X_test, num_iteration=model.best_iteration
    )

    y_preds.append(y_pred)
    models.append(model)

/opt/conda/lib/python3.9/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[LightGBM] [Info] Number of positive: 34219, number of negative: 45781
[LightGBM] [Info] Total Bins 1057
[LightGBM] [Info] Number of data: 80000, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.427737 -> initscore=-0.291088
[LightGBM] [Info] Start training from score -0.291088
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.568317	valid_1's binary_logloss: 0.569169
[20]	training's binary_logloss: 0.521052	valid_1's binary_logloss: 0.522385
[30]	training's binary_logloss: 0.499966	valid_1's binary_logloss: 0.501964
[40]	training's binary_logloss: 0.490119	valid_1's binary_logloss: 0.492759
[50]	training's bi

In [37]:
from sklearn.metrics import accuracy_score

y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

0.77203

In [38]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [39]:
sub = pd.read_csv('../data/tabular_playground/sample_submission.csv')

In [40]:
sub["Survived"] = y_sub
sub.to_csv("./subs/lightgbm_skfold_isAlone_categorized.csv",index=False)

In [41]:
sub.head()

,PassengerId,Survived
0,100000,0
1,100001,0
2,100002,1
3,100003,0
4,100004,1


# Pseudolabellingを使ってみる
- 参考
    - https://www.kaggle.com/c/tabular-playground-series-apr-2021/discussion/231738

In [42]:
test.loc[:,"Survived"] = y_sub

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [43]:
test

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Counter_Name,Counter_FirstName,Counter_SurName
0,0,3,0,19.0,0,0,63.01,0,1,1,1,18,1384
1,0,3,1,53.0,0,0,5.81,0,1,1,1,108,125
2,1,1,1,19.0,0,0,38.91,1,1,1,1,511,244
3,0,2,0,25.0,0,0,12.93,0,1,1,1,51,770
4,1,1,1,17.0,0,2,26.89,1,3,0,1,8,387
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,3,1,27.0,0,0,10.12,2,1,1,1,34,4
99996,0,1,0,59.0,1,0,68.31,0,2,0,2,1352,316
99997,0,3,0,47.0,0,0,10.87,0,1,1,1,8,52
99998,1,1,1,49.0,1,2,29.68,1,4,0,1,12,575


In [44]:
pseudoLabeled_train = pd.concat([train, test[:int(len(test) / 5)]], sort=False)
pseudoLabeled_train = pseudoLabeled_train.reset_index(drop=True)
pseudoLabeled_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Counter_Name,Counter_FirstName,Counter_SurName
0,1.0,1,0,31.00,2,0,27.14,0,3,0,1,47,51
1,0.0,3,0,31.00,0,0,13.35,0,1,1,1,54,31
2,0.0,3,0,0.33,1,2,71.29,0,4,0,2,165,1128
3,0.0,3,0,19.00,0,0,13.04,0,1,1,2,53,4736
4,1.0,3,0,25.00,0,0,7.76,0,1,1,1,39,3662
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,0.0,3,0,42.00,0,0,14.14,0,1,1,1,22,3302
119996,0.0,3,0,22.00,1,0,12.14,0,2,0,1,6,1085
119997,0.0,3,0,21.00,1,0,7.70,0,2,0,7,462,3302
119998,1.0,1,1,18.00,1,2,31.65,0,4,0,1,20,239


In [45]:
y_train = pseudoLabeled_train["Survived"]
X_train = pseudoLabeled_train.drop("Survived", axis=1)
X_test = test.drop("Survived", axis=1)

In [46]:
y_train

0         1.0
1         0.0
2         0.0
3         0.0
4         1.0
         ... 
119995    0.0
119996    0.0
119997    0.0
119998    1.0
119999    0.0
Name: Survived, Length: 120000, dtype: float64

In [47]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Counter_Name,Counter_FirstName,Counter_SurName
0,3,0,19.0,0,0,63.01,0,1,1,1,18,1384
1,3,1,53.0,0,0,5.81,0,1,1,1,108,125
2,1,1,19.0,0,0,38.91,1,1,1,1,511,244
3,2,0,25.0,0,0,12.93,0,1,1,1,51,770
4,1,1,17.0,0,2,26.89,1,3,0,1,8,387
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3,1,27.0,0,0,10.12,2,1,1,1,34,4
99996,1,0,59.0,1,0,68.31,0,2,0,2,1352,316
99997,3,0,47.0,0,0,10.87,0,1,1,1,8,52
99998,1,1,49.0,1,2,29.68,1,4,0,1,12,575


In [48]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Counter_Name,Counter_FirstName,Counter_SurName
0,1,0,31.00,2,0,27.14,0,3,0,1,47,51
1,3,0,31.00,0,0,13.35,0,1,1,1,54,31
2,3,0,0.33,1,2,71.29,0,4,0,2,165,1128
3,3,0,19.00,0,0,13.04,0,1,1,2,53,4736
4,3,0,25.00,0,0,7.76,0,1,1,1,39,3662
...,...,...,...,...,...,...,...,...,...,...,...,...
119995,3,0,42.00,0,0,14.14,0,1,1,1,22,3302
119996,3,0,22.00,1,0,12.14,0,2,0,1,6,1085
119997,3,0,21.00,1,0,7.70,0,2,0,7,462,3302
119998,1,1,18.00,1,2,31.65,0,4,0,1,20,239


In [49]:
y_preds = []
models = []
oof_train = np.zeros((len(X_train), ))
cv2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [50]:
for fold_id, (train_index, valid_index) in enumerate(cv2.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index,:]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(
        X_tr, y_tr, categorical_feature=categorical_features
    )

    lgb_eval = lgb.Dataset(
        X_val, y_val, reference=lgb_train, categorical_feature=categorical_features 
    )

    model = lgb.train(
        params, lgb_train, valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10, num_boost_round=1000, early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(
        X_val, num_iteration=model.best_iteration
    )

    y_pred = model.predict(
        X_test, num_iteration=model.best_iteration
    )

    y_preds.append(y_pred)
    models.append(model)

/opt/conda/lib/python3.9/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[LightGBM] [Info] Number of positive: 38868, number of negative: 57132
[LightGBM] [Info] Total Bins 1063
[LightGBM] [Info] Number of data: 96000, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.404875 -> initscore=-0.385193
[LightGBM] [Info] Start training from score -0.385193
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.541688	valid_1's binary_logloss: 0.54142
[20]	training's binary_logloss: 0.486324	valid_1's binary_logloss: 0.486013
[30]	training's binary_logloss: 0.461419	valid_1's binary_logloss: 0.461368
[40]	training's binary_logloss: 0.449598	valid_1's binary_logloss: 0.450004
[50]	training's bin

In [51]:

y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

0.8084333333333333

In [52]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)

sub["Survived"] = y_sub
sub.to_csv("./subs/lightgbm_skfold_pseudo_labeled.csv",index=False)